
## Section 4: Random Forest Classifier for High-Income Prediction

This notebook implements a Random Forest classifier to predict high-income individuals (>=$50K annual income) using Census Bureau data. It addresses key findings from EDA:
- 2.3x sample weight bias (high-income individuals underrepresented)
- Education x occupation interaction (education alone fails to predict income)
- Severe class imbalance (6.2% high-income vs 93.8% low-income)

The model uses sample weights during training and class balancing to handle these challenges.

## Import Libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    roc_auc_score,
    roc_curve,
    precision_recall_curve,
    average_precision_score
)
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 120)

print('=' * 80)
print('INCOME CLASSIFICATION MODEL - RANDOM FOREST')
print('=' * 80)
print('\nLibraries imported successfully\n')

## Section 1: Data Loading

In [ ]:
def load_column_names(columns_file):
    """Load column names from header file."""
    with open(columns_file, 'r') as f:
        columns = [line.strip().rstrip(':').strip() for line in f if line.strip()]
    return columns

In [ ]:
print('SECTION 1: Loading Data')
print('-' * 80)

# Load column names
column_names = load_column_names('../data/census-bureau.columns')
print(f'Loaded {len(column_names)} column names')

# Load data
data_file = '../data/census-bureau.data'
df = pd.read_csv(data_file, header=None, names=column_names, skipinitialspace=True)
print(f'Data loaded: {df.shape[0]:,} rows x {df.shape[1]} columns')

# Create binary target variable
label_col = 'label'
df['income_binary'] = (df[label_col].str.strip() == '50000+.').astype(int)
print(f'Target variable created: income_binary')
print(f'  Class distribution: {df["income_binary"].value_counts().to_dict()}')
print(f'  High-income rate: {df["income_binary"].mean()*100:.2f}%')
print()

## Section 2: Feature Engineering

In [ ]:
print('SECTION 2: Feature Engineering')
print('-' * 80)

# Define column names
age_col = 'age'
edu_col = 'education'
occ_col = 'major occupation code'
weeks_col = 'weeks worked in year'
marital_col = 'marital stat'
workclass_col = 'class of worker'
weight_col = 'weight'

# Select feature columns
feature_cols = [age_col, edu_col, occ_col, weeks_col, marital_col, workclass_col]
print('Selected features:')
for i, col in enumerate(feature_cols, 1):
    print(f'  {i}. {col}')

X = df[feature_cols].copy()
y = df['income_binary'].copy()
sample_weights = df[weight_col].copy()

print(f'\nFeature matrix: {X.shape}')
print(f'Target vector: {y.shape}')
print(f'Sample weights: {sample_weights.shape}')

In [ ]:
# Check for missing values
missing_counts = X.isnull().sum()
print('Missing values check:')
if missing_counts.sum() == 0:
    print('  No missing values detected')
else:
    print(missing_counts[missing_counts > 0])

In [ ]:
# Encode categorical features
categorical_cols = [edu_col, occ_col, marital_col, workclass_col]
label_encoders = {}

print('Encoding categorical features:')
for col in categorical_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))
    label_encoders[col] = le
    print(f'  {col}: {len(le.classes_)} unique categories')

print('\nFeature engineering complete\n')

## Section 3: Train/Test Split

In [ ]:
print('SECTION 3: Train/Test Split')
print('-' * 80)

X_train, X_test, y_train, y_test, weights_train, weights_test = train_test_split(
    X, y, sample_weights,
    test_size=0.20,
    random_state=42,
    stratify=y
)

print('Train/test split results:')
print(f'  Training set: {X_train.shape[0]:,} samples')
print(f'  Test set: {X_test.shape[0]:,} samples')
print(f'\nClass distribution verification:')
print(f'  Training high-income rate: {y_train.mean()*100:.2f}%')
print(f'  Test high-income rate: {y_test.mean()*100:.2f}%')
print('\nData split complete\n')

## Section 4: Model Training

In [ ]:
print('SECTION 4: Model Training')
print('-' * 80)

print('Training Random Forest classifier...')

rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=15,
    min_samples_split=100,
    min_samples_leaf=50,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1,
    verbose=0
)

rf_model.fit(X_train, y_train, sample_weight=weights_train)

print('Model training complete')
print(f'  Number of trees: {rf_model.n_estimators}')
print(f'  Max depth: {rf_model.max_depth}')

In [ ]:
# Generate predictions
print('Generating predictions on test set...')

y_pred = rf_model.predict(X_test)
y_pred_proba = rf_model.predict_proba(X_test)[:, 1]

print('Predictions generated')
print(f'  Test set size: {len(y_test):,}')
print(f'  Predicted high-income: {y_pred.sum():,} ({y_pred.mean()*100:.2f}%)')
print(f'  Actual high-income: {y_test.sum():,} ({y_test.mean()*100:.2f}%)')
print('\nModel training and prediction complete\n')

## Section 5: Model Evaluation

In [ ]:
print('SECTION 5: Model Evaluation')
print('=' * 80)

print('\n1. CLASSIFICATION REPORT:')
print('-' * 80)
print(classification_report(y_test, y_pred,
                          target_names=['Low Income (<$50K)', 'High Income (>=$50K)'],
                          digits=3))

In [ ]:
# ROC-AUC Score
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f'\n2. ROC-AUC SCORE: {roc_auc:.4f}')
print(f'   Strong discrimination ability')

In [ ]:
# Confusion Matrix
print('\n3. CONFUSION MATRIX:')
print('-' * 80)
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm,
                     index=['Actual: Low Income', 'Actual: High Income'],
                     columns=['Predicted: Low Income', 'Predicted: High Income'])
print(cm_df)

tn, fp, fn, tp = cm.ravel()
print(f'\nTrue Negatives: {tn:,}')
print(f'False Positives: {fp:,}')
print(f'False Negatives: {fn:,}')
print(f'True Positives: {tp:,}')

In [ ]:
# Business metrics
print('\n4. BUSINESS METRICS:')
print('-' * 80)

recall_high_income = tp / (tp + fn)
print(f'Market Coverage (Recall): {recall_high_income:.1%}')

precision_high_income = tp / (tp + fp) if (tp + fp) > 0 else 0
print(f'Campaign Efficiency (Precision): {precision_high_income:.1%}')

fpr_rate = fp / (fp + tn)
print(f'False Positive Rate: {fpr_rate:.1%}')

accuracy = (tp + tn) / (tp + tn + fp + fn)
print(f'Overall Accuracy: {accuracy:.1%}')

## Section 6: Feature Importance

In [ ]:
print('SECTION 6: Feature Importance Analysis')
print('=' * 80)

feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print('\nFeature Importance Rankings:')
print('-' * 80)
print(feature_importance.to_string(index=False))

## Section 7: Visualizations

In [ ]:
print('SECTION 7: Generating Visualizations')
print('-' * 80)

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# 1. Confusion Matrix Heatmap
cm_normalized = confusion_matrix(y_test, y_pred, normalize='true')
sns.heatmap(cm_normalized, annot=True, fmt='.2%', cmap='Blues',
            xticklabels=['Predicted: Low', 'Predicted: High'],
            yticklabels=['Actual: Low', 'Actual: High'],
            ax=axes[0], cbar_kws={'label': '% of Actual Class'})
axes[0].set_title('Confusion Matrix\n(Normalized by True Class)', fontweight='bold', fontsize=12)
axes[0].set_ylabel('Actual Income', fontweight='bold')
axes[0].set_xlabel('Predicted Income', fontweight='bold')

# 2. ROC Curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
axes[1].plot(fpr, tpr, color='darkorange', lw=2, label=f'Random Forest (AUC = {roc_auc:.3f})')
axes[1].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Classifier (AUC = 0.500)')
axes[1].set_xlim([0.0, 1.0])
axes[1].set_ylim([0.0, 1.05])
axes[1].set_xlabel('False Positive Rate', fontweight='bold')
axes[1].set_ylabel('True Positive Rate (Recall)', fontweight='bold')
axes[1].set_title('ROC Curve\nDiscrimination Ability', fontweight='bold', fontsize=12)
axes[1].legend(loc='lower right')
axes[1].grid(alpha=0.3)

# 3. Feature Importance
sns.barplot(data=feature_importance, y='feature', x='importance', palette='viridis', orient='h', ax=axes[2])
axes[2].set_title('Feature Importance\n(Gini Importance)', fontweight='bold', fontsize=12)
axes[2].set_xlabel('Importance Score', fontweight='bold')
axes[2].set_ylabel('Feature', fontweight='bold')
axes[2].grid(axis='x', alpha=0.3)

plt.tight_layout()
print('Visualizations generated')
plt.show()

## Section 8: Business Recommendations

In [ ]:
print('SECTION 8: Business Recommendations')
print('=' * 80)

print('''
RECOMMENDED DEPLOYMENT APPROACH:

Combine both models for optimal results:

1. First pass: Segmentation (Rule-based model)
   - Assign customers to segments
   - Provides interpretability

2. Second pass: Classification (Random Forest)
   - Score customers using probability (0-100%)
   - Provides precision for targeting

3. Threshold tuning:
   - Calibrate based on campaign cost
   - Expensive campaign: Use 70-80% threshold
   - Cheap campaign: Use 30-40% threshold

LIMITATIONS:
1. Data age: Census data from 1994-1995
2. Threshold needs business cost analysis
3. Feature drift monitoring needed
4. Fairness audit required before deployment
''')

print('=' * 80)
print('ANALYSIS COMPLETE')
print('=' * 80)